In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

flight_data = pd.read_csv('../input/flight-data-of-airline/flight_price.csv')
flight_data

In [ ]:
flight_data.head()

In [ ]:
flight_data.shape

In [ ]:
flight_data.isna().sum()

In [ ]:
flight_data.dropna(inplace = True)

In [ ]:
flight_data.isna().sum()

In [ ]:
flight_data.dtypes

In [ ]:
def change_into_datetime(col):
    flight_data[col] = pd.to_datetime(flight_data[col])   

In [ ]:
for i in {'Date_of_Journey','Dep_Time','Arrival_Time'}:
    change_into_datetime(i)

In [ ]:
flight_data.dtypes

In [ ]:
flight_data['journey_day'] = flight_data['Date_of_Journey'].dt.day
flight_data['journey_month'] = flight_data['Date_of_Journey'].dt.month

In [ ]:
flight_data.head()

In [ ]:
flight_data.drop('Date_of_Journey',axis=1,inplace=True)

In [ ]:
def extract_hour(df,col):
    df[col+'_hour'] = df[col].dt.hour

def extract_min(df,col):
    df[col+'_minute'] = df[col].dt.minute
    
def drop_column(df,col):
    df.drop(col,axis=1,inplace=True)   

In [ ]:
extract_hour(flight_data,'Dep_Time')
extract_min(flight_data,'Dep_Time')
drop_column(flight_data,'Dep_Time') 

In [ ]:
flight_data.head()

In [ ]:
extract_hour(flight_data,'Arrival_Time')
extract_min(flight_data,'Arrival_Time')
drop_column(flight_data,'Arrival_Time') 

In [ ]:
flight_data.head()

In [ ]:
flight_data.dtypes

In [ ]:
duration = list(flight_data['Duration'])
for i in range(len(duration)):
    if len(duration[i].split(' '))==2:
        pass
    else:
        if 'h' in duration[i]:
            duration[i] = duration[i] + ' 0m'
        else:
            duration[i]= '0h ' + duration[i]
    

In [ ]:
flight_data['Duration'] = duration

In [ ]:
flight_data.head()

In [ ]:
def hour(x):
    return x.split(' ')[0][:-1]
    
def minute(x):
    return x.split(' ')[1][:-1]

flight_data['Duration_hours'] = flight_data['Duration'].apply(hour)
flight_data['Duration_mins'] = flight_data['Duration'].apply(minute)

In [ ]:
flight_data.head()

In [ ]:
flight_data.drop('Duration',axis=1,inplace=True)

In [ ]:
flight_data.dtypes

In [ ]:
flight_data['Duration_hours'] = flight_data['Duration_hours'].astype(int)
flight_data['Duration_mins'] = flight_data['Duration_mins'].astype(int)

In [ ]:
flight_data.dtypes

In [ ]:
cat_col =[col for col in flight_data.columns if flight_data[col].dtype == 'O']
cat_col

In [ ]:
cont_col =[col for col in flight_data.columns if flight_data[col].dtype != 'O']
cont_col

In [ ]:
flight_data

In [ ]:
categorical = flight_data[cat_col]
categorical

In [ ]:
categorical.head()

In [ ]:
len(categorical['Airline'].unique())
Airline = pd.get_dummies(categorical['Airline'],drop_first = True)
Airline.head()

In [ ]:
len(categorical['Source'].unique())
Source = pd.get_dummies(categorical['Source'],drop_first = True)
Source.head()

In [ ]:
len(categorical['Destination'].unique())
Destination = pd.get_dummies(categorical['Destination'],drop_first = True)
Destination.head()

In [ ]:
categorical['Total_Stops'].value_counts()

In [ ]:
categorical['Total_Stops'].unique()

In [ ]:
dict = {'non-stop':0, '2 stops':2, '1 stop':1, '3 stops':3, '4 stops':4}

In [ ]:
categorical['Total_Stops']=categorical['Total_Stops'].map(dict)

In [ ]:
categorical.head()

In [ ]:
data_flight = pd.concat([categorical,Airline,Source,Destination,flight_data[cont_col]],axis=1)
data_flight.head()

In [ ]:
drop_column(data_flight,'Airline')
drop_column(data_flight,'Source')
drop_column(data_flight,'Destination')

In [ ]:
drop_column(data_flight,'Route')
drop_column(data_flight,'Additional_Info')

In [ ]:
data_flight.head()

In [ ]:
##finally data preprocessing is done ;) ;)


In [ ]:
X = data_flight.drop('Price',axis=1)
y = data_flight['Price']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
from sklearn import metrics

def predict(ml_model):
    model = ml_model.fit(X_train,y_train)
    print('Training score : ',model.score(X_train,y_train))
    
    y_prediction=model.predict(X_test)

    print('predictions are: \n',y_prediction)
    """
    print(X_test)
    print([[y_test]])
    print(y_prediction)
    """
    
    accuracy = metrics.r2_score(y_test,y_prediction)
    print('accuracy: ',accuracy)
    
    sns.distplot(y_test-y_prediction)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
predict(RandomForestRegressor())

In [ ]:
from sklearn.tree import DecisionTreeRegressor
predict(DecisionTreeRegressor())

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
predict(KNeighborsRegressor())

In [ ]:
from sklearn.linear_model import LinearRegression
predict(LinearRegression())